# Logistic Regression using Spark

## Binary Customer Churn

A marketing agency has many customers that use their service to produce ads for the client/customer websites. They've noticed that they have quite a bit of churn in clients. They basically randomly assign account managers right now, but want you to create a machine learning model that will help predict which customers will churn (stop buying their service) so that they can correctly assign the customers most at risk to churn an account manager. Creating a classification algorithm that will help classify whether or not a customer churned. Then the company can test this against incoming data for future customers to predict which customers will churn and assign them an account manager.

Here are the fields and their definitions:

    Name : Name of the latest contact at Company
    Age: Customer Age
    Total_Purchase: Total Ads Purchased
    Account_Manager: Binary 0=No manager, 1= Account manager assigned
    Years: Totaly Years as a customer
    Num_sites: Number of websites that use the service.
    Onboard_date: Date that the name of the latest contact was onboarded
    Location: Client HQ Address
    Company: Name of Client Company
    
The client wants to know which customers are most likely to churn given this data (they don't have the label yet).

In [2]:
import findspark
findspark.init('/home/ubuntu/spark-2.2.1-bin-hadoop2.7')

In [3]:
from pyspark.sql import SparkSession

In [4]:
spark = SparkSession.builder.appName('logregconsult').getOrCreate()

In [5]:
data = spark.read.csv('customer_churn.csv',inferSchema=True,
                     header=True)

In [6]:
data.printSchema()

root
 |-- Names: string (nullable = true)
 |-- Age: double (nullable = true)
 |-- Total_Purchase: double (nullable = true)
 |-- Account_Manager: integer (nullable = true)
 |-- Years: double (nullable = true)
 |-- Num_Sites: double (nullable = true)
 |-- Onboard_date: timestamp (nullable = true)
 |-- Location: string (nullable = true)
 |-- Company: string (nullable = true)
 |-- Churn: integer (nullable = true)



In [7]:
data.describe().show()

+-------+-------------+-----------------+-----------------+------------------+-----------------+------------------+--------------------+--------------------+-------------------+
|summary|        Names|              Age|   Total_Purchase|   Account_Manager|            Years|         Num_Sites|            Location|             Company|              Churn|
+-------+-------------+-----------------+-----------------+------------------+-----------------+------------------+--------------------+--------------------+-------------------+
|  count|          900|              900|              900|               900|              900|               900|                 900|                 900|                900|
|   mean|         null|41.81666666666667|10062.82403333334|0.4811111111111111| 5.27315555555555| 8.587777777777777|                null|                null|0.16666666666666666|
| stddev|         null|6.127560416916251|2408.644531858096|0.4999208935073339|1.274449013194616|1.764835592035

In [8]:
data.columns

['Names',
 'Age',
 'Total_Purchase',
 'Account_Manager',
 'Years',
 'Num_Sites',
 'Onboard_date',
 'Location',
 'Company',
 'Churn']

In [9]:
from pyspark.ml.feature import VectorAssembler

In [10]:
assembler = VectorAssembler(inputCols=['Age',
 'Total_Purchase',
 'Account_Manager',
 'Years',
 'Num_Sites'],outputCol='features')

In [11]:
output = assembler.transform(data)

In [12]:
final_data = output.select('features','churn')

### Test Train Split

In [13]:
train_churn,test_churn = final_data.randomSplit([0.7,0.3])

### Fit the model

In [14]:
from pyspark.ml.classification import LogisticRegression

In [15]:
lr_churn = LogisticRegression(labelCol='churn')

In [16]:
fitted_churn_model = lr_churn.fit(train_churn)

In [17]:
training_sum = fitted_churn_model.summary

In [18]:
training_sum.predictions.describe().show()

+-------+-------------------+------------------+
|summary|              churn|        prediction|
+-------+-------------------+------------------+
|  count|                609|               609|
|   mean|0.15435139573070608|0.1116584564860427|
| stddev|0.36158225205637606|0.3152046770264113|
|    min|                0.0|               0.0|
|    max|                1.0|               1.0|
+-------+-------------------+------------------+



### Evaluate results

Evaluate the results using the test data

In [19]:
from pyspark.ml.evaluation import BinaryClassificationEvaluator

In [20]:
pred_and_labels = fitted_churn_model.evaluate(test_churn)

In [21]:
pred_and_labels.predictions.show()

+--------------------+-----+--------------------+--------------------+----------+
|            features|churn|       rawPrediction|         probability|prediction|
+--------------------+-----+--------------------+--------------------+----------+
|[25.0,9672.03,0.0...|    0|[4.46704101673153...|[0.98864908414137...|       0.0|
|[26.0,8787.39,1.0...|    1|[0.06075427443807...|[0.51518389847911...|       0.0|
|[28.0,9090.43,1.0...|    0|[1.06597241979745...|[0.74383022718609...|       0.0|
|[29.0,8688.17,1.0...|    1|[2.31650335690200...|[0.91023464811032...|       0.0|
|[30.0,7960.64,1.0...|    1|[2.79183079299259...|[0.94223277581920...|       0.0|
|[30.0,8403.78,1.0...|    0|[5.76505355405716...|[0.99687457147418...|       0.0|
|[30.0,10183.98,1....|    0|[2.63949580393215...|[0.93336061099637...|       0.0|
|[30.0,13473.35,0....|    0|[2.78295114577519...|[0.94174755287424...|       0.0|
|[31.0,7073.61,0.0...|    0|[2.77054088549192...|[0.94106299306091...|       0.0|
|[31.0,9574.89,0

### Using AUC

In [22]:
churn_eval = BinaryClassificationEvaluator(rawPredictionCol='prediction',
                                           labelCol='churn')

In [23]:
auc = churn_eval.evaluate(pred_and_labels.predictions)

In [24]:
auc

0.7805851063829788

[Common question - what is a good AUC value?](https://stats.stackexchange.com/questions/113326/what-is-a-good-auc-for-a-precision-recall-curve)

### Predict on brand new unlabeled data

I still need to evaluate the new customers data!

In [25]:
final_lr_model = lr_churn.fit(final_data)

In [26]:
new_customers = spark.read.csv('new_customers.csv',inferSchema=True,
                              header=True)

In [27]:
new_customers.printSchema()

root
 |-- Names: string (nullable = true)
 |-- Age: double (nullable = true)
 |-- Total_Purchase: double (nullable = true)
 |-- Account_Manager: integer (nullable = true)
 |-- Years: double (nullable = true)
 |-- Num_Sites: double (nullable = true)
 |-- Onboard_date: timestamp (nullable = true)
 |-- Location: string (nullable = true)
 |-- Company: string (nullable = true)



In [28]:
test_new_customers = assembler.transform(new_customers)

In [29]:
test_new_customers.printSchema()

root
 |-- Names: string (nullable = true)
 |-- Age: double (nullable = true)
 |-- Total_Purchase: double (nullable = true)
 |-- Account_Manager: integer (nullable = true)
 |-- Years: double (nullable = true)
 |-- Num_Sites: double (nullable = true)
 |-- Onboard_date: timestamp (nullable = true)
 |-- Location: string (nullable = true)
 |-- Company: string (nullable = true)
 |-- features: vector (nullable = true)



In [30]:
final_results = final_lr_model.transform(test_new_customers)

In [31]:
final_results.select('Company','prediction').show()

+----------------+----------+
|         Company|prediction|
+----------------+----------+
|        King Ltd|       0.0|
|   Cannon-Benson|       1.0|
|Barron-Robertson|       1.0|
|   Sexton-Golden|       1.0|
|        Wood LLC|       0.0|
|   Parks-Robbins|       1.0|
+----------------+----------+



Ok! That is it! Now assign Acocunt Managers to Cannon-Benson,Barron-Robertson,Sexton-GOlden, and Parks-Robbins!